HDD Project

Raw data files needed to be converted to csv files for importation to pandas dataframes

In [ ]:
 #convert raw data file into csv
newrow = []

with open('geo/countyzones_split.csv', 'w') as f:
    with open('geo/countyzones_fixed.csv', 'r') as data:

        for line in data:

            lineParts = line[:2] + "," + line[2:]
            newrow = lineParts
            f.write(newrow)

Need to combine county data ID with States and population. Final output combined County Data including State, County 

In [ ]:
#combine population data with us counties. Limit to US States

import pandas as pd

convCSV = {}

#read csvs
CountyPops = pd.read_csv('output/newcountycsv.csv')
combinedZones = pd.read_csv('geo/countyzones_fixed.csv', sep=',', usecols=[0, 1])
stateCountyZones = pd.read_csv('geo/countyzones_split.csv', sep=',', usecols=[0, 1, 2])
StateCodes = pd.read_csv('geo/state_codes_fixed_conus.csv')

mergedCodes = stateCountyZones.merge(StateCodes, left_on='stateID', right_on='StateId')

mergedCodes['states_lower']=(mergedCodes['StateName'].str.lower()).str.strip()
CountyPops['states_lower2']= (CountyPops['State'].str.lower()).str.strip()
mergedCodes['counties_lower']=(mergedCodes['CountyName'].str.lower()).str.strip()
CountyPops['counties_lower2']=(CountyPops['countyName'].str.lower()).str.strip()

mergedPop = mergedCodes.merge(CountyPops, how='left', left_on=['counties_lower', 'states_lower'], right_on=['counties_lower2', 'states_lower2'])
final_df = pd.DataFrame(mergedPop, columns=['stateID', 'CountyID', 'states_lower', 'counties_lower', 'Pop'])

final_df = final_df[final_df.stateID < 57] # limits csv output to US States and exclude territories

print(final_df)

final_df.to_csv('combinedCountyData_conus.csv', index=False)

Find Centerpoint for each U.S. County

In [ ]:
#find center point of counties and merge with list. output final county data csv

import json
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon, shape

#initialize lists
result = []
gefsResult = []
dfCC = []
badCounties = []
  
# Opening county data JSON file
f = open('geo/gz_2010_is_050_00_20m2.json')

# reading gefs csv into dataframe
CountyData = pd.read_csv('combinedCountyData_conus.csv', dtype=str)
dfCountyData = pd.DataFrame(CountyData)

#convert state and county ids to strings with 2 and 3 chars respectively
CountyData['stateID'] = CountyData['stateID'].str.zfill(2)
CountyData['CountyID'] = CountyData['CountyID'].str.zfill(3)

data = json.load(f) #create json object
  
# Iterating through the json list
i=0
for Feature in data['features']:
    geoID = Feature['properties']['GEO_ID']
    countyID = Feature['properties']['COUNTY']
    stateID = Feature['properties']['STATE']
    countyName = Feature['properties']['NAME']
    coordsBoundary = (Feature['geometry']['coordinates'])
    coordsBoundaryFlat = [val for sublist in coordsBoundary for val in sublist]
    #find center of county polygon
    try:  
        centerCoord = Polygon(coordsBoundaryFlat).centroid
        lon = centerCoord.x
        lat = centerCoord.y
        newLine = [stateID, countyID, countyName, lat, lon]
        result.append(newLine)
    #if no valid polygon try multi-polygon solution    
    except: 
        try:
            polys = [x.buffer(0) for x in shape(Feature['geometry']).buffer(0).geoms] #extract shapely polygons into list of polygons
            centerCoord = MultiPolygon([*polys]).centroid # find center of polygons in list
            lon = centerCoord.x
            lat = centerCoord.y
            newLine = [stateID, countyID, countyName, lat, lon]
            result.append(newLine)
        # if no valid polygon or multipolygon geomtery found skip line and print error
        except:
            badLine = [stateID, countyID, countyName]
            print(f'shapely doesnt like {badLine}')
            badCounties.append(badLine)
    i = i + 1
  
# Closing file
f.close()

#creating county data dataframe with centroid lat lon
dfResults = pd.DataFrame(result, columns=['stateID','countyID','county','lat','lon'])
dfBadResults = pd.DataFrame(badCounties)

dfResults = pd.merge(dfResults, CountyData, how='left', left_on=['stateID', 'countyID'], right_on=['stateID', 'CountyID'] )
dfResults['Pop'] = (dfResults['Pop']).str.replace(',','')
dfResults['Pop'] = (dfResults['Pop']).truncate()
dfResults = dfResults.drop(columns=['CountyID'])

dfResults = dfResults[(dfResults.stateID).astype(int) < 57]

print(dfResults)
dfResults.to_csv('county_centroid_conus.csv', index=False) # final result

dfBadResults.to_csv('output/badCounties.csv', index=False) #csv for counties with failed center point calculation 

Assign GFS Grid Index value to Counties. Grid index references closest grid point coords to county center coords

In [ ]:
#create and join GFS GridIndex to County Data

from scipy.spatial.distance import cdist 
import numpy as np 
import pandas as pd

#read model grid csv and county data csv into dataframes  
gefsTemp = pd.read_csv('geo/gfstemp12_18.csv')
CountyData = pd.read_csv('county_centroid_conus.csv')

# creating gefs dataframe with appropriate columns
dfGefsTemp = pd.DataFrame(gefsTemp, columns=['value','latitude','longitude'])
dfGefsTemp.insert(0, 'gridIndex', range(1, 1 + len(dfGefsTemp)))
dfTempIndex = pd.DataFrame(dfGefsTemp, columns=['gridIndex', 'value'])

#find nearest grid point to county center
distArray = cdist(CountyData[['lat','lon']], dfGefsTemp[['latitude', 'longitude']])
gridIndex =  [dfGefsTemp["gridIndex"].iloc[np.argmin(x)] for x in distArray]

# insert gefs temp value column into county dataframe
CountyData['gridIndex'] = gridIndex
#left join gfs gridIndex to county list
CountyData = CountyData.merge(dfTempIndex, how='left', left_on=['gridIndex'], right_on=['gridIndex'])

CountyData['Pop'] = (CountyData['Pop']).truncate()   #truncate population column

CountyData.to_csv('county_grid_gfs.csv', index=False)   #create csv file from dataframe

Calculate degree hours from gfs csv output and assign grid point index to data points. Output is in csv format for manual add to analysis

In [ ]:
# join data from model grid points 

import pandas as pd

TempIndexList = []
inputFiles = []
runtime = 12 #label model runtime of gfs output
rundate = '112522' #lablek model runtime of gfs output

#create list of input files from gfs directory
def createInputFiles():
    i = 18
    while i < 43:
        istr = str(i).zfill(3)
        filename = f'gfs/gfs-{runtime}-fcsthr{istr}-{rundate}.csv'
        inputFiles.append(filename)
        i = i + 1
    return inputFiles

inputFiles = createInputFiles()

def gfsData(inputFiles):
    gridTempFrames = []
    TempIndexList = []
    i = 0
    j = 0
    l = 0

    def createGridFrame(inputfile, x):
        gridTempFrame = pd.read_csv(inputfile)
        gridTempFrames.append(gridTempFrame)
        return gridTempFrames

    while i < len(inputFiles):
        createGridFrame(inputFiles[i], i)
        i = i + 1

    def createIndex(df, y):
        df.insert(0, 'gridIndex', range(1, 1 + len(df)))
        dfTempIndex = pd.DataFrame(df, columns=['gridIndex', 'value', 'datetime'])
        dfTempIndex = dfTempIndex.rename(columns = {'value':f'tempK'})
        dfTempIndex = dfTempIndex.rename(columns = {'datetime':f'datetime'})
        TempIndexList.append(dfTempIndex)
        return TempIndexList

    while j < len(gridTempFrames):
        createIndex(gridTempFrames[j], j)
        j = j + 1
    
    #calculate heating and cooling degree hours
    def calc_dd(TempIndexList, x):
        TempIndexList[x]['hdd'] = (65 - (9/5*(TempIndexList[x]['tempK'] - 273) + 32).round())  
        TempIndexList[x]['cdd'] = ((9/5*(TempIndexList[x]['tempK'] - 273) + 32).round() - 65)
        TempIndexList[x].loc[TempIndexList[x][f'hdd'] < 0, 'hdd'] = 0
        TempIndexList.loc[TempIndexList['cdd'] < 0, 'cdd'] = 0
        TempIndexList[x].to_csv(f'gfs_output/gfs{x}.csv', index=False)
        return TempIndexList

    while l < len(TempIndexList):
        TempIndexList = calc_dd(TempIndexList, l)
        l = l + 1
    
    print(TempIndexList)
    return TempIndexList

TempIndexList = gfsData(inputFiles)

Insert GFS output for each forecast hour as table into RDS Postgres

In [ ]:
import psycopg2
from psycopg2 import extras
import boto3
import pandas as pd

TempIndexList = []
inputFiles = []
runtime = 12
rundate = '112522'

def createInputFiles():
    i = 18
    while i < 43:
        istr = str(i).zfill(3)
        filename = f'gfs/gfs-{runtime}-fcsthr{istr}-{rundate}.csv'
        inputFiles.append(filename)
        i = i + 1
    return inputFiles

inputFiles = createInputFiles()

def gfsData(inputFiles):
    gridTempFrames = []
    TempIndexList = []
    i = 0
    j = 0
    l = 0

    def createGridFrame(inputfile, x):
        gridTempFrame = pd.read_csv(inputfile)
        gridTempFrames.append(gridTempFrame)
        return gridTempFrames

    while i < len(inputFiles):
        createGridFrame(inputFiles[i], i)
        i = i + 1

    def createIndex(df, y):
        df.insert(0, 'gridIndex', range(1, 1 + len(df)))
        dfTempIndex = pd.DataFrame(df, columns=['gridIndex', 'value', 'datetime'])
        dfTempIndex = dfTempIndex.rename(columns = {'value':f'tempK{y}'})
        dfTempIndex = dfTempIndex.rename(columns = {'datetime':f'datetime{y}'})
        TempIndexList.append(dfTempIndex)
        return TempIndexList

    while j < len(gridTempFrames):
        createIndex(gridTempFrames[j], j)
        j = j + 1
    
    #calculate heating and cooling degree hours
    def calc_dd(TempIndexList, x):
        TempIndexList[x][f'hdd{x}'] = (65 - (9/5*(TempIndexList[x][f'tempK{x}'] - 273) + 32).round())  
        #CountyData['cdd'] = ((9/5*(CountyData[f'tempK{x}'] - 273) + 32).round() - 65)
        TempIndexList[x].loc[TempIndexList[x][f'hdd{x}'] < 0, f'hdd{x}'] = 0
        #CountyData.loc[CountyData['cdd'] < 0, 'cdd'] = 0
        return TempIndexList

    while l < len(TempIndexList):
        TempIndexList = calc_dd(TempIndexList, l)
        l = l + 1
    
    print(TempIndexList)
    return TempIndexList

TempIndexList = gfsData(inputFiles)

#Connect to RDS Postgres

ENDPOINT="{rds endpoint}"
PORT="{port}"
USER="{user}"
REGION="{region}"
DBNAME="{db name}"

session = boto3.Session(profile_name='dev')
client = session.client('rds', region_name='us-west-2')

token = client.generate_db_auth_token(DBHostname=ENDPOINT, Port=PORT, DBUsername=USER, Region=REGION)

try:
    conn = psycopg2.connect(host=ENDPOINT, port=PORT, database=DBNAME, user=USER, password=token, sslrootcert="SSLCERTIFICATE")
    cur = conn.cursor()
    cur.execute("""SELECT now()""")
    query_results = cur.fetchall()
    print(query_results)
except Exception as e:
    print("Database connection failed due to {}".format(e))     

#create tables and insert dataframes from tempindexlist
z = 18
w = 0
for df in TempIndexList:
    table = f'public.gfs{z}'
    tableName = f'gfs{z}'
    command = (f"""create table {table}(
    gridIndex int,
    tempK{w} numeric,
    datetime{w} timestamp,
    hdd{w} numeric)""") 
    


    cur.execute(command)

    def execute_values(conn, df, table):
        """
        Using psycopg2.extras.execute_values() to insert the dataframe
        """
        # Create a list of tupples from the dataframe values
        tuples = [tuple(x) for x in df.to_numpy()]
        # Comma-separated dataframe columns
        cols = ','.join(list(df.columns))
        # SQL quert to execute
        query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
        cursor = conn.cursor()
        try:
            extras.execute_values(cursor, query, tuples)
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        print("execute_values() done")
        cursor.close()

    execute_values(conn, df, tableName)
    # commit the changes
    conn.commit()
    z = z + 1
    w = w + 1

cur.close()

Inserting county data table with gfs gridIndex values to RDS. New gridIndex needs to be created for different model data or for any model boundary modification. 

In [ ]:
df = pd.read_csv('county_grid_gfs.csv', usecols=['stateID','countyID','county','lat','lon','states_lower','counties_lower','Pop','gridIndex'])

commandCreateCountyDataTable = ("""create table public.countyData_gfs(
stateID varchar,
countyID varchar,
county varchar,
lat float,
lon float,
states_lower varchar,
counties_lower varchar,
Pop numeric,
gridIndex numeric
)""") 

cur.execute(commandCreateCountyDataTable)

def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

execute_values(conn, df, 'countyData_gfs')